In [18]:
import pandas as pd

import sys
sys.path.append('../..')

from src.support_model import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
df = pd.read_csv("../../datos/Iteration1/prepped_data.csv", index_col=0)

Lo primero de todo es separar nuestros datos en VR y VP.

In [20]:
X = df.drop(columns = ["price"]).drop(columns=df.filter(like="has3D").columns)
y = df["price"]

Definimos la muestra para test y train.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Vamos a probar directamente con un gradient boost. En otras iteraciones se realizarán pruebas con otro tipo de modelos.

In [22]:
params_gb = {'n_estimators': [x for x in range(10,60,10)],
            'max_depth': [x for x in range(1,9)],
            'min_samples_split': [x for x in range(3,6)],
            'min_samples_leaf': [x for x in range(3,6)],
            'max_leaf_nodes': [x for x in range(3,11)]}

In [23]:
grid_search_gb = create_model(params=params_gb, X_train=X_train, y_train=y_train, method = GradientBoostingRegressor(random_state=42, learning_rate=0.1), cv = 5)


Fitting 5 folds for each of 2880 candidates, totalling 14400 fits


c:\Python311\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [24]:
grid_search_gb.best_params_

{'max_depth': 3,
 'max_leaf_nodes': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 3,
 'n_estimators': 40}

In [25]:
gb_estimator = grid_search_gb.best_estimator_

Vemos que hay mucho overfitting, por lo que habrá que hacer cambios para gestionarlo.

In [26]:
y_test_pred = gb_estimator.predict(X_test)
y_train_pred = gb_estimator.predict(X_train)

pd.DataFrame(metricas(y_train, y_train_pred, y_test, y_test_pred)).T

,r2_score,MAE,MSE,RMSE
train,0.778103,31.368822,1555.534205,39.440261
test,0.385046,42.479739,3460.591234,58.826790


Para ver dónde podemos cambiar cosas, podemos obtener las importancias de las variables.

In [28]:
pd.DataFrame(gb_estimator.feature_importances_, index=X.columns, 
             columns=["Importance"]).sort_values(by = "Importance",ascending=False)

,Importance
distance,0.536206
size,0.223603
province,0.106891
floor_2,0.027893
rooms_1,0.022452
floor_unknown,0.016819
floor_1,0.016137
hasLift_unknown,0.014643
rooms_2,0.009608
exterior_False,0.006225


## Conclusiones

Para siguientes iteraciones, eliminar algunas de las VP con importancia nula. Probar más modelos.